## Clean/transform game_officials_bronze -> game_officials_silver

##### 1. Import and load table

In [1]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F
from pyspark.sql import functions as F, Window

# Read a Delta table into a Spark DataFrame
df = spark.read.format("delta").load("Tables/game_officials_bronze")

StatementMeta(, 0389fab5-e21b-47bd-8eba-2c90bca3a120, 3, Finished, Available, Finished)

##### 2. Clean and transform data

In [2]:
# 1. Remove duplicate rows with exact information
df = df.dropDuplicates()

# 2. Function to convert camelCase / PascalCase to snake_case
def to_snake_case(name):
    return ''.join(['_' + c.lower() if c.isupper() else c for c in name]).lstrip('_')

# Apply to all columns in one line
df = df.toDF(*[to_snake_case(c) for c in df.columns])

# 3: Cast data types
df = (
    df  # ✅ Start from df, not df_clean
    .withColumn("game_id", F.col("game_id").cast("string"))
)

"""
# Show results
display(df.limit(10))

rows_final = df.count()
cols_final = len(df.columns)
print("")
print("Number of rows:", rows_final)
print("Number of columns:", cols_final)
"""

StatementMeta(, 0389fab5-e21b-47bd-8eba-2c90bca3a120, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bc81255d-87d4-402a-9aea-6e26b7d30b8d)


Number of rows: 95188
Number of columns: 3


##### 3. Load data to silver table

In [3]:
# Incoming Bronze dataframe
source_df = df  
key_col = "game_id"

# Path to Silver Lakehouse Delta table
target_path = "abfss://dc478dd4-e53e-4f21-add0-2e376dc173fe@onelake.dfs.fabric.microsoft.com/ce7ef0e9-78af-44db-b5ee-839dcf1c9e98/Tables/game_officials_silver"

if DeltaTable.isDeltaTable(spark, target_path):
    # Load existing target table
    existing_df = spark.read.format("delta").load(target_path).select(key_col).distinct()

    # Keep only new keys
    new_rows_df = source_df.join(existing_df, on=key_col, how="left_anti")

    if new_rows_df.limit(1).count() > 0:
        (new_rows_df.write
            .format("delta")
            .mode("append")
            .save(target_path))
        print(f"✅ Appended {new_rows_df.count()} new rows to game_officials_silver in Lakehouse_Silver.")
    else:
        print("No new rows to append. game_officials_silver is already up to date.")
else:
    # First load → create the Silver table
    (source_df.write
        .format("delta")
        .mode("overwrite")
        .save(target_path))
    print(f"✅ Initial load complete: created game_officials_silver in Lakehouse_Silver with {source_df.count()} rows.")


StatementMeta(, 0389fab5-e21b-47bd-8eba-2c90bca3a120, 5, Finished, Available, Finished)

No new rows to append. game_officials_silver is already up to date.


In [1]:
"""
# Show first 10 rows
display(df.limit(10))

# Check shape of data
rows = df.count()
cols = len(df.columns)
print("")
print("Number of rows:", rows)
print("Number of columns:", cols)
print("")

# Check for nulls
null_counts = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df.columns
])
null_counts.show()

# Check for exact duplicate rows
dup_count = df.groupBy(df.columns).count().filter(F.col("count") > 1).count()
print("Number of exact duplicated rows:", dup_count)
print("")

# Check schema data type
df.printSchema()
"""

StatementMeta(, 0230b79e-f437-47ac-8861-ac63f8a571c6, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2253f1b7-9684-45df-9ca6-dfd60df716da)


Number of rows: 105853
Number of columns: 3

+-------+-------------+-------------+
|game_id|official_name|official_type|
+-------+-------------+-------------+
|      0|            0|            0|
+-------+-------------+-------------+

Number of exact duplicated rows: 10663

root
 |-- game_id: integer (nullable = true)
 |-- official_name: string (nullable = true)
 |-- official_type: string (nullable = true)



##### 2. Check duplicated rows

In [2]:
"""
# Create a window partitioned by all columns
w = Window.partitionBy(df.columns)

# Add a count of how many times each row appears
duplicates_all = (
    df.withColumn("dup_count", F.count("*").over(w))
      .filter(F.col("dup_count") > 1)   # keep only duplicates
)

display(duplicates_all)
"""

StatementMeta(, 7a7ee9c3-8abf-4265-ac75-d639745080cf, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3e333113-dfbb-4ce8-a9bb-a2c119144275)

##### 3. Cleaned/transformed to silver